In [1]:
%%time
import pandas as pd
import numpy as np
data=pd.read_csv(r'D:\Projects\Goods_prediction\real_data\data2019.csv',decimal=',',
                dtype={'DiscountCardId':np.int32,'goodsid':np.int32,'GoodsGroup3Id':np.int16,
                        'Warehouseid':np.int16,'creditgroupid':np.int64,'Quantity':np.float32,'summ':np.float32},sep=';')
data=data.drop(['GoodsGroup2Id'],axis=1)
warehouse=pd.read_csv(r'D:\Projects\Goods_prediction\real_data\warehousetype.csv',sep=';',names=['Warehouseid','WarehouseTypeId'],
                     dtype={'WarehouseTypeId':np.int16})
data=data.merge(warehouse,how='left',on='Warehouseid')
data['CreditGroupTime']=pd.to_datetime(data['CreditGroupTime'])
data['weekday']=data['CreditGroupTime'].dt.dayofweek
data['weekend']=(data['CreditGroupTime'].dt.weekday >=5).astype(np.int16)
data['hour']=data['CreditGroupTime'].dt.hour
data['day']=data['CreditGroupTime'].dt.day
data['week']=data['CreditGroupTime'].dt.week
#data['weekofmonth']=data['CreditGroupTime'].dt.week/(data['CreditGroupTime'].dt.month+1)
goods=pd.read_csv(r'D:\Projects\Goods_prediction\real_data\goods.csv',sep=';',names=['GoodsGroup3Id'])
data=data.merge(goods.reset_index(drop=False),how='left',on='GoodsGroup3Id')
data=data.drop(['GoodsGroup3Id'],axis=1)
important_gds=[27,51,57,58,72,75,113,240]
data=data.rename(columns={'index':'GoodsGroup3Id'})
#data=data.dropna().reset_index(drop=True)
data['GoodsGroup3Id']=data['GoodsGroup3Id'].astype(np.int32)
data=data.sort_values('creditgroupid').reset_index(drop=True)

Wall time: 8min 1s


In [2]:
%%time
from scipy.stats.mstats import mode
def freq(data):
    freq=mode(data.values)
    return freq[0]
def get_new_columns(name,aggs):
    itog=[]
    for k in aggs.keys():
        for agg in aggs[k]:
            if isinstance(agg,str):
                itog.append(name + '_' + k + '_' + agg)
            else:
                itog.append(name + '_' + k + '_' + agg.__name__)
    return itog
def second_freq(data):
    try:
        freq=data.value_counts().index[1]
    except IndexError:
        freq=None
    return freq
def third_freq(data):
    try:
        freq=data.value_counts().index[2]
    except IndexError:
        freq=None
    return freq
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
def all_features(data):
    args={'creditgroupid':['nunique'],
          'Warehouseid':['nunique'],
  #        'WarehouseTypeId':[freq],
          'goodsid':['count'],
          'GoodsGroup3Id':[freq,second_freq],
          'summ':['sum','mean'],
          'CreditGroupTime':['last'],
#          'weekday':[freq,second_freq],
          'weekend':['mean'],
          'hour':['std','max','min'],
          'day':['mean'],
          'week':['nunique']
          
         }
    last_data=data['CreditGroupTime'].max()
    new_columns=get_new_columns('all',args)
    data=data.groupby('DiscountCardId').agg(args)
    data.columns=new_columns
    data.reset_index(drop=False,inplace=True)
    data['all_goods_check']=data['all_goodsid_count']/data['all_creditgroupid_nunique']
    data['all_summ_check']=data['all_summ_sum']/data['all_creditgroupid_nunique']
    data['last_day_purchase']=(last_data-data['all_CreditGroupTime_last']).dt.days
    data=data.drop(['all_goodsid_count','all_summ_sum','all_CreditGroupTime_last'],axis=1)
    return data
def month_features(data):
    last_data=data['CreditGroupTime'].max()
    data=data[data['CreditGroupTime']>(last_data-pd.DateOffset(months=1))].reset_index(drop=True)
    args={'creditgroupid':['nunique'],
          'Warehouseid':['nunique'],
#          'WarehouseTypeId':[freq],
          'goodsid':['count'],
           'Quantity':['max'],
          'GoodsGroup3Id':[freq,second_freq],
          'summ':['sum','mean'],
          'weekend':['mean'],
          'weekday':[freq],
          'hour':['max','min','std'],
          'week':['nunique']
          
         }
    new_columns=get_new_columns('month',args)
    data=data.groupby('DiscountCardId').agg(args)
    data.columns=new_columns
    data.reset_index(drop=False,inplace=True)
    data['month_goods_check']=data['month_goodsid_count']/data['month_creditgroupid_nunique']
    data['month_summ_check']=data['month_summ_sum']/data['month_creditgroupid_nunique']
    data=data.drop(['month_goodsid_count','month_summ_sum'],axis=1)
    return data
def last_features(data):
    lst=data.groupby('DiscountCardId')['creditgroupid'].max()
    lst.columns=['lst_check']
    lst=lst.reset_index(drop=False)
    data=lst.merge(data,how='left',on=['creditgroupid','DiscountCardId'])
    args={'WarehouseTypeId':['max'],
          'goodsid':['count'],
          'weekday':['max'],
          'summ':['sum']
        
         }
    new_columns=get_new_columns('lst',args)
    data=data.groupby('DiscountCardId').agg(args)
    data.columns=new_columns
    data.reset_index(drop=False,inplace=True)
#    data=data.drop(['month_goodsid_count','month_summ_sum','month_CreditGroupTime_last'],axis=1)
#    data=data.merge(twogroup,how='left',on='DiscountCardId')
    return data


Wall time: 1.03 s
Parser   : 323 ms


In [3]:
%%time
from collections import Counter
def goods_label(data):
    itog=np.zeros(len(goods))
    itog[np.unique(data)]=1
    return [itog]
def goods_count(data):
    itog=np.zeros(len(goods))
    c=dict(Counter(data))
    itog[list(c)]=list(c.values())
    return [itog]
def label_features(data):
    
    ddata=data.groupby(['DiscountCardId'])['GoodsGroup3Id'].max()
    ddata=ddata.reset_index(drop=False)
    ddata.rename(columns={'GoodsGroup3Id':'target'},inplace=True)
    ddata['target']=(ddata['target']>0).astype(np.int16)
#    data=data.groupby(['DiscountCardId'])['day'].agg('max')
#    data=data.reset_index(drop=False)
#    data.rename(columns={'day':'dataoftarget'},inplace=True)
#    data=data.merge(ddata,on='DiscountCardId',how='left')
 #   data['target']=np.where(data['target'].values>0,1,0)
    return ddata
def goods_features(data):
    last_data=data['CreditGroupTime'].max()
    args={'creditgroupid':['nunique'],
          'Warehouseid':['nunique'],
          'WarehouseTypeId':[freq],
          'goodsid':['count'],
          'summ':['sum','mean'],
          'Quantity':['mean','std'],
          'weekend':['mean'],
          'weekday':[freq],
          'hour':['max','min',freq,'std'],
          'CreditGroupTime':['last']
         }
    new_columns=get_new_columns('all_goods',args)
    ddata=data.groupby(['DiscountCardId','GoodsGroup3Id']).agg(args)
    ddata.columns=new_columns
    ddata.reset_index(drop=False,inplace=True)
    ddata['all_goods_goods_check']=ddata['all_goods_goodsid_count']/ddata['all_goods_creditgroupid_nunique']
    ddata['all_goods_summ_check']=ddata['all_goods_summ_sum']/ddata['all_goods_creditgroupid_nunique']
    ddata['goods_last_day_purchase']=(last_data-ddata['all_goods_CreditGroupTime_last']).dt.days
    ddata=ddata.drop(['all_goods_goodsid_count','all_goods_summ_sum','all_goods_CreditGroupTime_last'],axis=1)
    data=data[data['CreditGroupTime']>(last_data-pd.DateOffset(months=1))].reset_index(drop=True)
    args={'creditgroupid':['nunique'],
          'Warehouseid':['nunique'],
          'WarehouseTypeId':[freq],
          'goodsid':['count'],
          'summ':['sum','mean'],
          'Quantity':['max','mean'],
          'hour':['max','min',freq,'std']
         }
    new_columns=get_new_columns('month_goods',args)
    data=data.groupby(['DiscountCardId','GoodsGroup3Id']).agg(args)
    data.columns=new_columns
    data.reset_index(drop=False,inplace=True)
    data['month_goods_goods_check']=data['month_goods_goodsid_count']/data['month_goods_creditgroupid_nunique']
    data['month_goods_summ_check']=data['month_goods_summ_sum']/data['month_goods_creditgroupid_nunique']
    data=data.drop(['month_goods_goodsid_count','month_goods_summ_sum'],axis=1)
    ddata=ddata.merge(data,how='left',on=['DiscountCardId','GoodsGroup3Id'])
    return ddata
def goods_features(data):
    data=data.groupby(['DiscountCardId'])['GoodsGroup3Id'].agg(goods_count)
    data=data.reset_index(drop=False)
    data.rename(columns={'GoodsGroup3Id':'data'},inplace=True)
    return data
def goods_days(data):
    itog=np.zeros(len(goods))
    c=dict(Counter(data))
    itog[list(c)]=list(c.values())
    return [itog]
def df_dates(data):
    itog=[]
    for i in np.array(data):
        if len(i)==1:
            continue
        else:
            a=np.array(np.diff(i), dtype='timedelta64[D]')
            
            a=[int(i/ np.timedelta64(1, 'D')) for i in a]
            itog.append(np.squeeze(mode(a)[0][0]))
    if len(itog)==0:
        md=-1
    else:
        md=mode(itog)[0]
    return md
def most_days_goods_features(data):
    aggs={
        'CreditGroupTime':['max']
    }
    mx=data['CreditGroupTime'].max()
    data=data.drop_duplicates(subset=['DiscountCardId','creditgroupid','GoodsGroup3Id']).groupby(['DiscountCardId','GoodsGroup3Id'])['CreditGroupTime'].agg(lambda x:list(x))

    data=data.reset_index(drop=False,inplace=False)
    data=data.groupby(['GoodsGroup3Id'])['CreditGroupTime'].agg(df_dates)
    data=data.reset_index(drop=False,inplace=False)

    return data
def days_goods_features(data):
    aggs={
        'CreditGroupTime':['max']
    }
#     data=data.pivot_table(index='DiscountCardId',columns='GoodsGroup3Id',aggfunc=aggs)
    mx=data['CreditGroupTime'].max()
    data=data[data['GoodsGroup3Id'].isin(important_gds)]
    result=most_days_goods_features(data)
    data=data.groupby(['DiscountCardId','GoodsGroup3Id'])['CreditGroupTime'].agg('last')

    data=data.reset_index(drop=False,inplace=False)
    data['CreditGroupTime']=(mx-data['CreditGroupTime']).dt.days
    data['CreditGroupTime']=data['CreditGroupTime'].replace({0:1})
    data=data.pivot_table(index='DiscountCardId',columns='GoodsGroup3Id',aggfunc=aggs)
    data=result['CreditGroupTime'].values/data
    data=data.reset_index(drop=False)
#    ind=[n for _,_,n in data.columns.tolist()]
#    a=np.ones((len(data.index),len(goods)))*999
    data=data.fillna(999)
    return data
def dataoftarget(data):
    dd=(data['CreditGroupTime'].max()).day
    if dd<=8:
        dd=0
    elif dd<=15:
        dd=1
    elif dd<=22:
        dd=2
    else:
        dd=3
    return dd

Wall time: 0 ns


In [13]:
%%time
import datetime
base_date=pd.Timestamp(2018,9,3)
goods=pd.read_csv(r'D:\Projects\Goods_prediction\real_data\goods.csv',sep=';',names=['GoodsGroup3Id'])
main_data=reduce_mem_usage(data)
inputs=range(52)
def result(i):
    data_period=base_date+pd.DateOffset(weeks=i)
    start_data=data_period-pd.DateOffset(months=3)
    label_period=base_date+pd.DateOffset(weeks=i+1)
    label=main_data[(main_data['CreditGroupTime']>=data_period)&(main_data['CreditGroupTime']<=label_period)].reset_index(drop=True)
    data=main_data[(main_data['CreditGroupTime']>start_data)&(main_data['CreditGroupTime']<data_period)].reset_index(drop=True)
    
#     main=pd.DataFrame({'DiscountCardId':np.repeat(data.DiscountCardId.unique(),len(goods)),
#                        'GoodsGroup3Id':np.tile(goods.GoodsGroup3Id.values,len(data.DiscountCardId.unique()))})
    new_data=all_features(data)
    new_data=new_data.merge(month_features(data),how='left',on='DiscountCardId')
    new_data=new_data.merge(last_features(data),how='left',on='DiscountCardId')
    new_data=reduce_mem_usage(new_data)
    main=new_data.copy()
    del new_data
 #   new_data=goods_features(data)
 #   new_data=pd.DataFrame.from_records(new_data['data'].apply(np.squeeze).values)
 #   main=main.join(new_data)
 #   del new_data
    
#    new_data=days_goods_features(data)
#    new_data=pd.DataFrame.from_records(new_data)
#    main=main.join(new_data,rsuffix='_timeforgoods')
#    del new_data
    
    main['diff_freq']=(main['all_GoodsGroup3Id_freq']==main['month_GoodsGroup3Id_freq']).astype(int)
    main['diff_second_freq']=(main['all_GoodsGroup3Id_second_freq']==main['month_GoodsGroup3Id_second_freq']).astype(int)
    main['diff_summ_check']= main['month_summ_check']/main['all_summ_check']
#    main=main.merge(goods_features(data),how='left',on=['DiscountCardId','GoodsGroup3Id'])
    label=label_features(label)
    main=main.merge(label,how='left',on='DiscountCardId')
    main['dayoftarget']=dataoftarget(data)
    
    dem=pd.read_csv(r'D:\Projects\Goods_prediction\real_data\demographic2019.csv',
                sep=';',names=['DiscountCardId','whatsex','family','age']).merge(pd.read_csv(r'D:\Projects\Goods_prediction\real_data\for Hleb.csv',sep=';'),
                                                                                how='left',on='DiscountCardId')
    dem.rename(columns={'(No column name)':'timex'},inplace=True)
    main=main.merge(dem,how='left',on='DiscountCardId')
    main['timex']=pd.to_datetime(main['timex'])
    mx=data['CreditGroupTime'].max()
    main['timex']=(mx-main['timex']).apply(lambda x : x.days)
#    main['dataoftarget']=main['dataoftarget'].fillna(int(main['dataoftarget'][~main['dataoftarget'].isna()].mean()))
    main=main.fillna(0)
    main=reduce_mem_usage(main)
    main.to_hdf(f'D:\Projects\\Goods_prediction\\real_data\\Dat2\\data{i}.h5','data')
    return print(f'{i} is ready!')
from joblib import Parallel, delayed
import multiprocessing
par = Parallel(n_jobs=-12)(delayed(result)(i) for i in inputs)
print(par)

Mem. usage decreased to 8322.00 Mb (0.0% reduction)
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Wall time: 1h 13min 39s


Mem. usage decreased to 8322.00 Mb (48.6% reduction)
Mem. usage decreased to 13.79 Mb (67.8% reduction)
Mem. usage decreased to 16.38 Mb (35.4% reduction)
Wall time: 6min 22s


,DiscountCardId,all_creditgroupid_nunique,all_Warehouseid_nunique,all_GoodsGroup3Id_freq,all_GoodsGroup3Id_second_freq,all_summ_mean,all_weekend_mean,all_hour_std,all_hour_max,all_hour_min,...,lst_summ_sum,diff_freq,diff_second_freq,diff_summ_check,target,dayoftarget,whatsex,family,age,timex
0,510745,18,1,75,21.0,2.767578,0.062500,2.646484,21,12,...,7.859375,1,0,0.609375,1.0,0,1,4,48,-915
1,510749,25,3,33,89.0,2.380859,0.430176,2.978516,19,10,...,23.984375,0,0,1.077148,0.0,0,1,2,67,-275
2,510757,19,3,178,82.0,2.300781,0.279053,4.343750,22,10,...,17.281250,0,0,0.892090,1.0,0,0,7,50,5
3,510792,26,3,240,214.0,2.107422,0.470215,2.042969,18,10,...,9.429688,0,0,0.897461,0.0,0,0,3,32,-74
4,510811,59,4,89,72.0,3.029297,0.364014,3.019531,22,9,...,45.031250,1,1,1.166016,1.0,0,0,4,35,-578
5,510813,1,1,75,214.0,1.461914,1.000000,0.000000,9,9,...,7.308594,1,1,1.000000,0.0,0,0,6,48,-562
6,510841,8,2,63,63.0,1.330078,0.532227,2.660156,20,12,...,15.226562,0,0,0.947266,0.0,0,0,3,38,-224
7,510857,7,1,70,113.0,2.285156,0.285645,3.500000,19,11,...,15.132812,0,0,0.000000,1.0,0,0,2,59,-577
8,510862,1,1,49,191.0,1.464844,0.000000,0.000000,19,19,...,24.906250,1,1,1.000000,0.0,0,1,1,37,-646
9,510870,7,6,21,240.0,3.792969,0.177368,2.638672,16,10,...,1.179688,0,0,0.035126,0.0,0,1,3,46,-557


In [1]:
%%time
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
def data_loading(binary='No'):
    for n,i in tqdm_notebook(enumerate([0,1,2,4,5,7,8,10,11,14,15,16,18,19,20,21,23,25,26,28,29,30,31,32,34,36,37,38,40,43,44,45,47,48,50])):
        if n==0:
            train=pd.read_hdf(f'D:\Projects\Goods_prediction\\real_data\Dat2\data{i}.h5','data')
        else:
            train=train.append(pd.read_hdf(f'D:\Projects\Goods_prediction\\real_data\Dat2\data{i}.h5','data'))
    train=reduce_mem_usage(train)
    for n,i in tqdm_notebook(enumerate([3,6,9,12,13,17,22,24,27,33,35,39,41,42,46,49,51])):
        if n==0:
            test=pd.read_hdf(f'D:\Projects\Goods_prediction\\real_data\Dat2\data{i}.h5','data')
        else:
            test=test.append(pd.read_hdf(f'D:\Projects\Goods_prediction\\real_data\Dat2\data{i}.h5','data'))
    goods=pd.read_csv('D:\Projects\Goods_prediction\\real_data\goods.csv',names=['GoodsGroup3Id'],sep=';')
    if binary=='No':
        train=train[train['target']!=0].reset_index(drop=True)
        train=reduce_mem_usage(train)
        test=test[test['target']!=0].reset_index(drop=True)
        test=reduce_mem_usage(test)
        
        target=train['target'].values
        train_target=[]
        for i in range(len(target)):
            if isinstance(target[i],int):
                train_target.append(np.zeros(len(goods)))
            else:
                train_target.append(target[i][0])        
        target=test['target'].values
        test_target=[]
        for i in range(len(target)):
            if isinstance(target[i],int):
                test_target.append(np.zeros(len(goods)))
            else:
                test_target.append(target[i][0])
    else:
        train.loc[train['target']!=0,'target']=1
        test.loc[test['target']!=0,'target']=1
        train_target=train['target'].values
        test_target=test['target'].values
    return train,train_target,test,test_target
train,train_target,test,test_target=data_loading(binary='yes')


Mem. usage decreased to 646.40 Mb (0.0% reduction)



Wall time: 36.3 s


In [9]:
import gc
gc.collect()
cat=['dayoftarget','age','all_GoodsGroup3Id_freq','all_GoodsGroup3Id_second_freq','diff_freq','diff_second_freq','month_GoodsGroup3Id_freq','month_GoodsGroup3Id_second_freq','month_weekday_freq']
import lightgbm as lgb
param = {'objective': 'binary', 
    'metric': ['binary_error','auc'],
            'boosting_type': 'gbdt', 
            'n_jobs': -1, 'max_depth': -1, 
            'n_estimators': 3500, 
#            'subsample_freq': 2, 
 #           'min_data_per_group': 100, 
#            'max_cat_to_onehot': 4, 
#            'cat_l2': 10.0, 
#            'cat_smooth': 10.0, 
#            'max_cat_threshold': 32, 
            'metric_freq': 10, 
            'verbosity': -1, 
#            'colsample_bytree': 0.5, 
            'learning_rate': 0.0081033234451294376, 
            'min_child_samples': 20, 
            'min_child_weight': 6, 
            'min_split_gain': 0.06, 
            'num_leaves': 100, 
            'reg_alpha': 34, 
            'reg_lambda': 1, 
#            'subsample': 0.9,
        'first_metric_only':True}
#stop_columns=train.iloc[:,36:265].columns.tolist()+train.iloc[:,-4:].columns.tolist()
#stop_columns=train.iloc[:,-4:].columns.tolist()
df_train_columns = [c for c in train.columns if c not in ['target','DiscountCardId']]
val_pr=np.zeros(len(test))

train_data=lgb.Dataset(train[df_train_columns],label=np.asarray(train_target),categorical_feature=cat)
val_data=lgb.Dataset(test[df_train_columns],label=np.asarray(test_target),categorical_feature=cat)

#del train;
num_round = 10000
model=lgb.train(param,train_data,num_round,valid_sets=[train_data,val_data],verbose_eval=100,early_stopping_rounds=300)
val_pr=model.predict(test[df_train_columns],num_iteration=model.best_iteration)

Training until validation scores don't improve for 300 rounds.
[100]	training's binary_error: 0.269028	training's auc: 0.779192	valid_1's binary_error: 0.279614	valid_1's auc: 0.778366
[200]	training's binary_error: 0.25631	training's auc: 0.78022	valid_1's binary_error: 0.265289	valid_1's auc: 0.77921
[300]	training's binary_error: 0.253994	training's auc: 0.781121	valid_1's binary_error: 0.262242	valid_1's auc: 0.779859
[400]	training's binary_error: 0.253085	training's auc: 0.781901	valid_1's binary_error: 0.261268	valid_1's auc: 0.780322
[500]	training's binary_error: 0.25263	training's auc: 0.782597	valid_1's binary_error: 0.260907	valid_1's auc: 0.780675
[600]	training's binary_error: 0.252228	training's auc: 0.783249	valid_1's binary_error: 0.260707	valid_1's auc: 0.780923
[700]	training's binary_error: 0.251823	training's auc: 0.783849	valid_1's binary_error: 0.260597	valid_1's auc: 0.781094
[800]	training's binary_error: 0.251427	training's auc: 0.784405	valid_1's binary_error

In [14]:
%%time
import gc
gc.collect()
from catboost import Pool,CatBoostClassifier
df_train_columns = [c for c in train.columns if c not in ['target','DiscountCardId']]

#df_train_columns = [c for c in train.columns if c not in ['target','DiscountCardId',('CreditGroupTime', 'max', 161)]]
val_pr=np.zeros(len(test))

train_data=Pool(train[df_train_columns],label=np.asarray(train_target),cat_features=cat)
val_data=Pool(test[df_train_columns],label=np.asarray(test_target),cat_features=cat)

#del train;
num_round = 10000
model=CatBoostClassifier(iterations=400, learning_rate=0.01,metric_period=10,early_stopping_rounds=50,loss_function='CrossEntropy',eval_metric='AUC')
model.fit(train_data,eval_set=val_data,use_best_model=True)
val_pr=model.predict(test[df_train_columns],prediction_type='Probability')

0:	test: 0.7601791	best: 0.7601791 (0)	total: 6.25s	remaining: 41m 34s
10:	test: 0.7703274	best: 0.7703274 (10)	total: 1m 6s	remaining: 38m 54s
20:	test: 0.7715425	best: 0.7715425 (20)	total: 2m 2s	remaining: 36m 54s
30:	test: 0.7725007	best: 0.7725007 (30)	total: 2m 59s	remaining: 35m 30s
40:	test: 0.7729981	best: 0.7729981 (40)	total: 3m 58s	remaining: 34m 50s
50:	test: 0.7734026	best: 0.7734026 (50)	total: 4m 55s	remaining: 33m 41s
60:	test: 0.7738544	best: 0.7738544 (60)	total: 5m 50s	remaining: 32m 26s
70:	test: 0.7740555	best: 0.7740555 (70)	total: 6m 45s	remaining: 31m 19s
80:	test: 0.7744443	best: 0.7744443 (80)	total: 7m 40s	remaining: 30m 12s
90:	test: 0.7746547	best: 0.7746547 (90)	total: 8m 35s	remaining: 29m 10s
100:	test: 0.7749022	best: 0.7749022 (100)	total: 9m 30s	remaining: 28m 8s
110:	test: 0.7750620	best: 0.7750620 (110)	total: 10m 24s	remaining: 27m 5s
120:	test: 0.7752570	best: 0.7752570 (120)	total: 11m 20s	remaining: 26m 9s
130:	test: 0.7754858	best: 0.7754858 (

KeyboardInterrupt: 

In [11]:
#val_pr[:,1]
#test['target']
#(test['target'].values==np.where(val_pr>0.5,1,0)).mean()
#tar=pd.DataFrame(test['target'].values,columns=['target'])
#tar['pr']=np.where(val_pr>0.6,1,0)
#tar[tar['target']==1]['pr'].mean()
feat=pd.DataFrame(model.feature_importance(),columns=['Importance'])
feat['val']=model.feature_name()
feat.sort_values('Importance',ascending=False)


In [2]:
%%time
df_train_columns = [c for c in train.columns if c not in ['target','DiscountCardId']]
cat=['dayoftarget','age','all_GoodsGroup3Id_freq','all_GoodsGroup3Id_second_freq','diff_freq','diff_second_freq','month_GoodsGroup3Id_freq','month_GoodsGroup3Id_second_freq','month_weekday_freq']
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
def optim_lgb(max_depth,num_leaves,lambda_l1,lambda_l2,min_split_gain,min_child_weight,bagging_fraction):
    train_data=lgb.Dataset(train[df_train_columns],label=np.asarray(train_target),categorical_feature=cat)
    val_data=lgb.Dataset(test[df_train_columns],label=np.asarray(test_target),categorical_feature=cat)
    def run_lgb(max_depth,num_leaves,lambda_l1,lambda_l2,min_split_gain,min_child_weight,bagging_fraction):
        num_round = 10000
        params = {
                    "objective" : "binary",
                    "metric" : ['binary_error','auc'], 
                    "boosting": "gbdt",
                    "learning_rate" : 0.01,
                    'n_estimators': 1000,
                     'max_depth':int(round(max_depth)),
                     'num_leaves':int(num_leaves),
                     'lambda_l1':max(lambda_l1, 0),
                     'lambda_l2':max(lambda_l2, 0),
                     'min_split_gain':min_split_gain,
                     'min_child_weight':min_child_weight,
                     'bagging_fraction':max(min(bagging_fraction, 1), 0)
                }
        model=lgb.train(params,train_data,num_round,valid_sets=[train_data,val_data],verbose_eval=False,early_stopping_rounds=300)

        pred_val = model.predict(test[df_train_columns], num_iteration=model.best_iteration)

    #    return pred_val
        return  pred_val
    val_pr=run_lgb(max_depth,num_leaves,lambda_l1,lambda_l2,min_split_gain,min_child_weight,bagging_fraction)
    return roc_auc_score(test_target,val_pr)
from bayes_opt import BayesianOptimization
optimizer = BayesianOptimization(optim_lgb,{'max_depth':(-1,25),'num_leaves':(15,100),'lambda_l1':(0,80),'lambda_l2':(0,80),
'min_split_gain':(0.001, 0.1),'min_child_weight':(5,50),'bagging_fraction':(0.8,1)}
    
)
optimizer.maximize(init_points=3,
    n_iter=30)
print(optimizer.max)

|   iter    |  target   | baggin... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------


C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


|  1        |  0.7815   |  0.8976   |  69.48    |  79.64    |  23.84    |  13.15    |  0.07689  |  97.66    |


C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


|  2        |  0.7757   |  0.8712   |  8.053    |  27.51    |  2.418    |  21.98    |  0.05468  |  40.04    |


...

|  30       |  0.7814   |  0.8884   |  2.343    |  64.62    |  24.98    |  6.249    |  0.09954  |  99.0     |


C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


|  31       |  0.7815   |  0.9435   |  74.31    |  74.98    |  24.79    |  46.13    |  0.01464  |  99.88    |


C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


KeyboardInterrupt: 

In [11]:
#from sklearn.externals import joblib
#joblib.dump(model, 'LGBM.pkl')
model

array([0.64293959, 0.89604436, 0.886869  , ..., 0.93284839, 0.87675115,
       0.89972404])